In [ ]:
from elements.yolo import LINE_DETECTION
import numpy as np
import cv2
import os
from glob import glob
from time import time
from skimage.feature import canny
from skimage.transform import probabilistic_hough_line
from copy import deepcopy

In [ ]:
def rad(degree):
    return (degree * np.pi/180)

In [ ]:
def line_detector_func(image, theta_range=None):
    line_image = 0 * image
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # canny edge detector filter
    edges = canny(image_gray,sigma=sigma,low_threshold=low_threshold,high_threshold=high_threshold)
    lines = probabilistic_hough_line(edges,
                                     threshold=threshold,
                                     line_length=max(image_gray.shape)//4,
                                     line_gap=max(image_gray.shape),
                                     theta=theta_range,
                                     seed=2022)

    idx = 0
    while len(lines)>1 and idx<len(lines)-1:
        (x00, y00), (x01, y01) = lines[idx]
        (x10, y10), (x11, y11) = lines[idx+1]
        m0 = (y01-y00) / (x01-x00)
        m1 = (y11-y10) / (x11-x10)
        if np.abs(m1-m0) <= tetha_thr:
            lines.pop(idx+1)
        else:
            idx+=1

    for ((x0, y0), (x1, y1)) in lines: 
        cv2.line(line_image, (x0, y0), (x1, y1), (255, 255, 255), thickness=thickness)
        
    return line_image, edges, lines

In [ ]:
def same_region(region, row, col):
    for i in range(len(region)):
    r, c = region[i]
    if (row==r and col==c+1) or (row==r+1 and (col==c-1 or col==c or col==c+1)):
        return True

def get_regions(line_image):
    regions = {}
    for row in range(line_image.shape[0]):
        for col in range(line_image.shape[1]):
            if line_image[row,col] == 255:
                # regions dict is empty
                if len(regions) == 0:
                    regions[0] = [(row,col)]

                # regions dict in not empty
                else:
                    flag = 0
                    for reg  in regions.keys():
                        if flag == 1:
                            break
                        if same_region(regions[reg], row, col):
                            regions[reg].append((row,col))
                            flag = 1
                        
                    if flag == 0:
                        regions[len(regions)] = [(row,col)]
  
    return regions

In [ ]:
def get_middle(region, top):
    iter=0
    if top == True:
        for i in range(len(region)-1):
            if region[i][0] == region[i+1][0]:
                iter+=1
            else:
                return iter+1
    
    else:
        for i in range(1,len(region)-1):
            if region[-1*i][0] == region[-1*(i+1)][0]:
                iter += 1
            else:
                return -1*(iter+1)

In [ ]:
def generate_line(regions, image_height):
    slopes = np.zeros(shape=[len(regions)])
    intercepts = np.zeros(shape=[len(regions)])
    line_data = np.zeros(shape=[len(regions), 2])

    for reg in regions.keys():
        (x0, y0) = regions[reg][0]
        (x1, y1) = regions[reg][1]
        m = (y1-y0) / (x1-x0)
        slopes[reg] = m
        intercepts[reg] = ((y1-x1*m)+(y0-x0*m))/2

    intercepts = image_height*np.around(intercepts/image_height, decimals=1)
    slopes = np.around(slopes, decimals=1)

    line_data = np.transpose([slopes, intercepts])
    unique_lines, count = np.unique(line_data, return_counts=True, axis=0)
  
    return unique_lines[np.argsort(slopes)]

In [ ]:
def inner_point(p1, p2, height, width):
    (yf,xf) = p2
    (yi,xi) = p1
    height = height+1
    width = width+1
    if yi<height and xi<width:
        return (yi,xi)
    elif yf<height and xf<width:
        return (yf,xf)
    else:
        print("no inner point possible")

In [ ]:
def compelete_lines(line_array, h, w):
    points = []
    h=h-1
    w=w-1

    for m,b in line_array:
        if b>h:
            if m<0:
                y0,x0 = h,(h-b)/m
                y1,x1 = 0,(-b/m)
                y2,x2 = (m*w+b),w
                y1,x1 = inner_point((y1,x1),(y2,x2),h,w)
            else:
                print("b>h m>=0 impossible")
            
        elif b<0:
            if m>=0:
                y0,x0 = 0,(-b/m)
                y1,x1 = h,(h-b)/m
                y2,x2 = (m*w+b),w
                y1,x1 = inner_point((y1,x1),(y2,x2),h,w)
            else:
                print("b<0 m<0 impossible")
            
        else:
            if m<0:
                y0,x0 = b,0
                y1,x1 = 0,(-b/m)
                y2,x2 = (m*w+b),w
                y1,x1 = inner_point((y1,x1),(y2,x2),h,w)
            else:
                y0,x0 = b,0
                y1,x1 = h,(h-b)/m
                y2,x2 = (m*w+b),w
                y1,x1 = inner_point((y1,x1),(y2,x2),h,w)

        points.append([y0,x0,y1,x1])

    points=np.asarray(points,dtype='uint64')
    return points

### Hyperparameters

In [ ]:
theta = np.concatenate([np.arange(start=rad(-60),stop=rad(-5),step=rad(1)),np.arange(start=rad(5),stop=rad(60),step=rad(1))])
tetha_thr = 0.5
thickness = 2
region_thr = 2000
# canny hyoerparameters
sigma = 4
low_threshold = 1
high_threshold = 25
# probabilistic_hough_line hyperparameters
threshold = 20

### Violations

In [ ]:
# for line_sebghat viokation
line_num = [0,1]

### Image Path

In [ ]:
image = './mask.jpg'
# load image
frame = cv2.imread(image)

### Detection Process

In [ ]:
# detector objects
line_detector = LINE_DETECTION('weights/line_model.pt')

Fusing layers... 


In [ ]:
# detection process, detecting lines
lines = line_detector.detect(frame)

In [ ]:
# create mask from line bboxes
bb_mask = np.zeros((frame.shape))

for line in lines:
  [(xmin,ymin),(xmax,ymax)] = line['bbox']
  bb_mask = cv2.rectangle(bb_mask, (xmin,ymin), (xmax,ymax), [255,255,255] , -1)
bb_mask = bb_mask.astype('int')

In [ ]:
# extract edges from whole image
line_image, edges, lines = line_detector_func(frame,theta_range=theta)

In [ ]:
# extract edges and lines from lines only
bb_edges = edges * bb_mask[:,:,0]
bb_edges_3d = np.stack((bb_edges,)*3, axis=-1).astype('uint8')

line_image, edges, lines = line_detector_func(bb_edges_3d,theta_range=theta)

In [ ]:
# apply dilation and erosion on extracted lines
kernel = np.ones((5,5),np.uint8)
dilation = cv2.dilate(line_image,kernel,iterations = 5)
erosion = cv2.erode(dilation,kernel,iterations = 5)

line_image = erosion[(erosion.shape[0]//2)-100:(erosion.shape[0]//2)+100,:,0]

In [ ]:
regions = get_regions(line_image)

In [ ]:
i = 0
regions_final = {}
for reg in regions.keys():
    if len(regions[reg]) >= region_thr:
        idx_top = get_middle(regions[reg], top=True)//2
        idx_bot = get_middle(regions[reg], top=False)//2
        regions_final[i] = [(regions[reg][idx_top][1], regions[reg][idx_top][0]+(frame.shape[0]//2)-100), (regions[reg][idx_bot][1], regions[reg][idx_bot][0]+(frame.shape[0]//2)-100)]
        i+=1

In [ ]:
lines = generate_line(regions_final, frame.shape[0])

In [ ]:
height, width = frame.shape[:2]  
points = compelete_lines(line_array=lines, h=height, w=width)

In [ ]:
# generate mask from road
mask_img = np.zeros(list(edges.shape))
a = mask_img.shape

(y0, x0, y1, x1) = points[line_num[0]]
y1 = int(y1)
y0 = int(y0)
x1 = int(x1)
x0 = int(x0)
m1 = (x1-x0)/(y1-y0)
b1 = ((x0-y0*m1)+(x1-y1*m1))/2

(y0, x0, y1, x1) = points[line_num[1]]
y1 = int(y1)
y0 = int(y0)
x1 = int(x1)
x0 = int(x0)
m2 = (x1-x0)/(y1-y0)
b2 = ((x0-y0*m2)+(x1-y1*m2))/2

for xx in range(a[0]):
    for yy in range(a[1]): 
        if (-yy+xx*m1+b1)<0 and (-yy+xx*m2+b2)>0:
            mask_img[xx,yy]=255

In [ ]:
# save mask
cv2.imwrite('output_mask.jpg', cv2.resize(mask_img, (mask_img.shape[1], mask_img.shape[0])))

True